In [9]:
#import various python libraries
import requests
import csv
import bs4
import re
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
from datetime import date
today = date.today()

In [13]:
# Load Data Files
df = pd.read_csv('../data/CA_Coronovirus_Websites_by_county.csv')

In [14]:
# Scraper helper function
def regex_paragraph_1(list_level_1):  
    '''
    Makes a 'paragraph' of text from a list in order to perform REGEX..
    Returns a text of combined strings for parsing. 
    '''
    paragraph = ''
    for x in list_level_1: 
            paragraph += x + ' '
    return(paragraph)


In [15]:
# SAVE YESTERDAYS COLUMN NAMES AND TIMESTAMPS

def save_yesterday_data (cases=None, deaths=None, tests=None, recovered_p=None):
    if tests:
        yesterday_tests = tests
    if cases:
        yesterday_cases = cases
    if deaths:
        yesterday_deaths = deaths
    if recovered_p:
        yesterday_recovered = recovered
    return(yesterday_cases,yesterday_deaths,yesterday_tests,yesterday_recovered)

y_c, y_d, y_t, y_r = save_yesterday_data(cases=name, deaths=string_, tests=test_admin,recovered_p=recovered)

# Initial Current TImestamp columns to yesterday values
df[name] = df[y_c]
df[string_] = df[y_d]
df[recovered] = df[y_r]
df[test_admin] = df[y_t]

In [16]:
# Make Columns with CURRENT TIMESTAMP
test_admin = str(today) +" Tests" 
name = str(today)+' Confirmed Cases'
string_ = str(today) +' Deaths' 
recovered = str(today)+ ' Recovered'


In [22]:
# Function that checks website accuracy and does initial soup work
def initial_study(county_no):
    print(df.loc[county_no])
    print(df.loc[county_no,'Website'])
    
    # Optional Soup Error Checking
    # soup_c = soups[county_no].find_all('div')
    # for x in soup_c:
        # print(x)
    
    

In [23]:
initial_study(3)

County                                        Berkeley
Website    https://www.cityofberkeley.info/coronavirus
Name: 3, dtype: object
https://www.cityofberkeley.info/coronavirus


In [24]:
# A beautiful soup web scraping function
def run_bs4(link, lxml=None):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
    headers = {'User-Agent': user_agent}
    page = requests.get(link)
    html = page.content
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return(soup)

In [25]:
# SCRAPES COUNTY PUBLIC HEALTH WEBSITES
soups= []
for x in df['Website']:
    s = run_bs4(x)
    soups.append(s)

## BERKLEY


In [735]:
for x in soups[3].find_all('p'):
    if 'cases' in str(x):
        try:
            text = x.find('strong').find(text=True)
        except:
            pass
df.loc[3, name]= text
df.loc[3]

County                                                           Berkeley
Website                       https://www.cityofberkeley.info/coronavirus
2020-04-02 Confirmed Cases                                             23
2020-04-02 Deaths                                                     NaN
2020-04-02 Tests                                                      NaN
2020-04-02 Recovered                                                  NaN
Name: 3, dtype: object

# Alameda

In [942]:
print(df.loc[0, 'Website'])

http://www.acphd.org/2019-ncov.aspx


In [943]:
with open('../webpages/Coronavirus Disease (COVID-19) - ACPHD.mhtml', 'r') as htm:
    soups[0] = htm.read()

In [944]:
cases = re.findall(r'Positive Cases: <em>([0-9]+)', soups[0])[0]

cases

'373'

In [945]:
deaths = re.findall(r'Deaths: <em>([0-9]+)', soups[0])[0]

deaths

'9'

In [946]:
df.loc[0, name]= cases
df.loc[0, string_] = deaths

## ALPINE

In [740]:
df.iloc[1]

County                                                            Alpine
Website                       http://alpinecountyca.gov/AlertCenter.aspx
2020-04-02 Confirmed Cases                                           NaN
2020-04-02 Deaths                                                    NaN
2020-04-02 Tests                                                     NaN
2020-04-02 Recovered                                                 NaN
Name: 1, dtype: object

In [741]:
df.iloc[1].fillna(0)

County                                                            Alpine
Website                       http://alpinecountyca.gov/AlertCenter.aspx
2020-04-02 Confirmed Cases                                             0
2020-04-02 Deaths                                                      0
2020-04-02 Tests                                                       0
2020-04-02 Recovered                                                   0
Name: 1, dtype: object

## Amador

In [742]:
#amador
paragraph = ''
for x in soups[2].find_all('span'):
    for y in x.find_all(text=True):
        paragraph +=  y.strip()+ ' '
    
    
# print(paragraph)    
cases = re.findall(r'Cases As of [A-Z][a-z]+ [0-9]+, [0-9]+: ([0-9]+)', paragraph)[0]
# print(cases)
df.loc[2, name] = cases
df.loc[2, name]

'3'

## BUTTE

In [743]:
df.loc[4]

County                                                                    Butte
Website                       http://www.buttecounty.net/publichealth/Home.aspx
2020-04-02 Confirmed Cases                                                   10
2020-04-02 Deaths                                                             0
2020-04-02 Tests                                                            NaN
2020-04-02 Recovered                                                        NaN
Name: 4, dtype: object

In [744]:
paragraph = ''
for no, x in enumerate(soups[4].find_all('td')):
    
    if 'Total' in str(x):
         paragraph += soups[4].find_all('td')[no+1].find(text=True).strip() + ' '
cases = re.findall(r'([0-9]+) ([0-9]+)', paragraph)[0]
df.loc[4,name] = re.findall(r'([0-9]+) ([0-9]+)', paragraph)[0][0]
df.loc[4, string_] = re.findall(r'([0-9]+) ([0-9]+)', paragraph)[0][1]

## Calaveras

In [745]:
for x in soups[5].find_all('span', class_= 'Head', id="dnn_ctr8879_dnnTITLE_titleLabel"):
    results = x.contents
    print(results)

case = re.findall(r'Number of confirmed cases in Calaveras County: ([0-9]+)', results[0])[0]
df.loc[5, name] = case

['Number of confirmed cases in Calaveras County: 3']


In [746]:
case

'3'

In [747]:
df.loc[5]

County                                               Calaveras
Website                       https://covid19.calaverasgov.us/
2020-04-02 Confirmed Cases                                   3
2020-04-02 Deaths                                          NaN
2020-04-02 Tests                                           NaN
2020-04-02 Recovered                                       NaN
Name: 5, dtype: object

## COLUSA

In [748]:
for x in soups[6].find_all('div', class_="fr-view"):
    for row in x.find_all('strong'):
        # print(row)
        if 'Cases' in str(row):
            cases = row.find(text=True)
cases = re.findall(r'Cases: ([0-9]+)', cases)[0]
df.loc[6, name]=cases

# CONTRA COSTA

In [749]:
paragraph = ' '
for x in soups[7].find_all('div', class_="txtNew"):
    for y in x.find_all('h1'):
        paragraph += y.find(text=True) + ' '
df.loc[7, string_] = re.findall(r'DEATHS ([0-9]+)', paragraph)[0]
df.loc[7, name] = re.findall(r'TOTAL CASES ([0-9]+)', paragraph)[0]

## DEL NORTE

In [750]:
print(df.loc[8, 'Website'])

http://www.co.del-norte.ca.us/departments/health-human-services/public-health


In [751]:
paragraph = ''
for x in soups[8].find_all('div'):
    for z in x.find_all(text=True):
            # print(z.strip())
            paragraph += z.replace('\xa0', '').strip() + ' '
            
tests = re.findall(r'Total Number of Tests Administered               ([0-9]+)', paragraph)[0] 
cases = re.findall(r'Number of Positive COVID-19 Cases               ([0-9]+)', paragraph)[0]

In [752]:
df.loc[8, name]= case
df.loc[8, test_admin]= tests

## EL DORADO

In [753]:
print(df.loc[9, "Website"])

https://www.edcgov.us/Government/hhsa/Pages/EDCCOVID-19-Cases.aspx


In [754]:
import re

In [755]:
paragraph_ = ''
for x in soups[9].find_all('table'):
    text = x.find_all(text=True)
    # print(text)
    
    for no, y in enumerate(text):
        paragraph_ += y.strip().replace('\n', '').replace('\u200b', '') +' '
    print(paragraph_)
    

            
cases = re.findall(r'Positive Tests ([0-9]+)', paragraph_)[0]
print(cases)
deaths =re.findall(r'Deaths ([0-9]+)', paragraph_)[0]
print(deaths)
cases = cases
tests = re.findall(r'Total Number of Tests\*\*  ([0-9]+)', paragraph_)[0]

df.loc[9, name] = cases
df.loc[9, string_] = deaths
df.loc[9, test_admin] = tests

Cases El Dorado County Residents   Positive Tests 18  Deaths 0   Total Number of Tests**  764   Negative Tests  596   Pending Tests  +/- 150 
18
0


## FRESNO

In [756]:
paragraph_case = ''
paragraph_deaths = ''
paragraph_tests = ''
for no, x in enumerate(soups[10].find_all('li')):
    text = x.find(text=True)
    # print(text)
    if 'cases' in text:
        paragraph_case += soups[10].find_all('li')[no+1].find(text=True) + ' '
        paragraph_case += soups[10].find_all('li')[no+2].find(text=True) + ' '
        paragraph_case += soups[10].find_all('li')[no+3].find(text=True) + ' '
        
    if 'deaths' in text:
        paragraph_deaths += text + ' '
    if 'Tests' in text:
        paragraph_tests += text + ' '
        
paragraph_tests = paragraph_tests.replace(',', '')
# print(paragraph_tests)
cases = re.findall(r'([0-9]+) \(Travel-Related\) ([0-9]+) \(Person-to-Person\) ([0-9]+) \(Community-Spread\)',\
                   paragraph_case) 
tests = re.findall(r'([0-9]+) \(Tests',paragraph_tests)[0]
cases = sum([int(x) for x in cases[0]])

# print(tests, cases)
        
df.loc[10, name] = cases
df.loc[10, test_admin] = tests

In [757]:
df.loc[10]

County                                                                   Fresno
Website                       https://www.co.fresno.ca.us/departments/public...
2020-04-02 Confirmed Cases                                                   45
2020-04-02 Deaths                                                           NaN
2020-04-02 Tests                                                           1650
2020-04-02 Recovered                                                        NaN
Name: 10, dtype: object

## GLENN

In [758]:
county = 11
print(df.loc[county, 'Website'])

https://www.countyofglenn.net/dept/health-human-services/public-health/covid19


In [759]:
df.iloc[11]

County                                                                    Glenn
Website                       https://www.countyofglenn.net/dept/health-huma...
2020-04-02 Confirmed Cases                                                    2
2020-04-02 Deaths                                                           NaN
2020-04-02 Tests                                                             32
2020-04-02 Recovered                                                        NaN
Name: 11, dtype: object

In [760]:
paragraph = ''
for x in soups[11].find_all('strong'):
    for y in x.find_all(text=True):
        paragraph += y.strip() + ' '
paragraph

'We understand that there may be concerns about Novel Coronavirus\xa0Disease\xa0also known as COVID-19. Glenn County HHSA’s Public Health Department is working closely with State and local\xa0partners and other healthcare systems to assess the situation locally and provide guidance as necessary.\xa0See below for COVID-19 testing results for the county. Updates: Update April 1, 2020 Glenn County COVID-19 Cases 2 How can you protect yourself and your family? "click here" "haga clic aquí" People at Increased Risk Who should be tested for COVID-19? Testing must be done by a healthcare provider. Glenn County Public Health cannot perform COVID-19 testing. Glenn County Public Health cannot perform COVID-19 testing. People who should be tested Call ahead so that you can be safely evaluated to prevent exposing others. Press Releases: Issued by Glenn County: Emergency Operations Center HHSA | Public Health Sheriff\'s Office Board of Supervisors '

In [761]:
df.loc[11, name] = re.findall(r'COVID-19 Cases ([0-9]+)', paragraph)[0]
df.loc[11, test_admin] = 32
                              

## Humboldt

In [762]:
df.loc[12, 'Website'] = 'https://humboldtgov.org/2707/COVID-Test-Results'
#'https://humboldtgov.org/CivicAlerts.aspx?AID=2657'

In [763]:
base = 'https://humboldtgov.org'

In [764]:
soups[12] = run_bs4(df.loc[12, 'Website']'')

SyntaxError: invalid syntax (<ipython-input-764-393c30412360>, line 1)

In [ ]:
new_link = []
for x in soups[12].find_all('div', class_="widgetBody cpGrid cpGrid1"):
    if len(x)>0:
        try:
            for y in x.find('li', class_='widgetItem'):
                try: 
                    new_link.append(y.find('a').get('href'))
                except:
                    pass
        except:
                pass

In [ ]:
new_link

In [ ]:
latest_link = base + new_link[0]
positive = []
for x in soups[12].find_all('div', class_="outer col col24 first last"):
    # print(x)
    for y in x.find_all('div'):
        text =y.find(text=True)
        #print(text)
        positive.append(str(text).replace('\n', "").replace('\t', "").replace('\r', ""))

In [ ]:
paragraph = regex_paragraph_1(positive)


In [ ]:
no_case = 0
cases = re.findall(r'Total [new]* positive cases [confirmed]* on [A-Z][a-z]+ [0-9]+: ([0-9]+) |Total positive cases [confirmed]* on [A-Z][a-z]+ [ ]*[0-9]+: [ ]*([0-9]+)', paragraph)
for x,y in zip(cases[:3], cases[3:]):
    print(x[0], y[1])
    no_case += int(x[0]) + int(y[1])

In [ ]:
df.loc[12, name] = no_case + 10
df.loc[12, test_admin] = 439 + 409

In [ ]:
df.loc[12].fillna(0)

## imperial

In [ ]:

print(df.loc[13, 'Website'])

In [ ]:
paragraph = ''
for x in soups[13].find_all('div', class_="panel-body"):
    #print(x)
    for y in x.find_all('td'):
        text = y.find(text=True)
        if text:
            paragraph += text.replace('\xa0', "") +' '

cases = re.findall(r'\(Confirmed Cases\) ([0-9]+)', paragraph)[0]
tests = re.findall(r'Total Tested ([0-9]+)', paragraph)[0]
df.loc[13, name] = cases
df.loc[13, test_admin]= tests

In [ ]:
paragraph

##  INYO

In [ ]:
try:
    paragraph = ''
    for no, x in enumerate(soups[14].find_all('meta')):
        if x:
            paragraph += str(x).strip() + ' '

        paragraph = paragraph.replace("\n", "").replace("\xa0", "").replace("\t", "")

    deaths= re.findall(r'([0-9]+) deaths', paragraph)[0]
    cases= re.findall(r'([0-9]+) confirmed', paragraph)[0]
    df.loc[14, name] = cases
    df.loc[14, string_] = deaths
except:
    pass

In [ ]:
print(df.loc[14, 'Website'])

In [ ]:
paragraph = ''
for no, x in enumerate(soups[14].find_all('table')):
    for y in x.find_all('li'):
        for z in y.find_all(text=True):
            # print(z)
            paragraph += str(z).strip() + ' '

mono = ''          
for no, x in enumerate(soups[14].find_all('p')):
    for y in x.find_all('span'):
        for z in y.find_all(text=True):
            mono += z.strip() + ' '

paragraph = paragraph.replace("\n", "").replace("\xa0", "").replace("\t", "")

deaths= re.findall(r'([0-9]+) deaths', paragraph)[0]
cases= re.findall(r'([0-9]+) confirmed', paragraph)[0]
tests = re.findall(r'Tests Administered: ([0-9]+)', paragraph)[0]
df.loc[14, name] = cases
df.loc[14, string_] = deaths
df.loc[14, test_admin] = tests

In [ ]:
mono_results = re.findall(r'Mono County Cases: ([0-9]+) confirmed COVID-19 case \| ([0-9]+) deaths', mono)
df.loc[27, name] = mono_results[0][0]
df.loc[27, string_] = mono_results[0][1]


In [ ]:
## KERN

In [947]:

print(df.loc[15, 'Website'])

with open('../webpages/2019 Novel Coronavirus - Kern County Public Health.mhtml', 'r') as htm:
        soups[15] = htm.read()
soups[15] = bs4.BeautifulSoup(soups[15])
    
df.loc[15, name] = 155
df.loc[15, string_] = 1
df.loc[15, test_admin] = 4110

https://kernpublichealth.com/2019-novel-coronavirus/


## Kings County

In [ ]:
paragraph =''
soups[16].find_all('b')
for x in soups[16].find_all('b'):
    for y in x.find_all('h3'):
        paragraph += str(y) + ' ' 
df.loc[16, name] = re.findall(r'Confirmed Cases: ([0-9]+)', paragraph)[0]
df.loc[16, test_admin] = re.findall(r'Samples Collected by Health Dept.: ([0-9]+)', paragraph)[0]

In [ ]:
paragraph

In [ ]:
re.findall(r'Confirmed Cases: ([0-9]+)', paragraph)[0]


In [ ]:
re.findall(r'Samples Collected by Health Dept.: ([0-9]+)', paragraph)[0]

## Lake

In [ ]:
county = 17

In [ ]:
print(df.loc[17, 'Website'])

In [ ]:
df.loc[county, test_admin] = 110

## LASSEN

In [ ]:
print(df.loc[18, 'Website'])

# LONG BEACH 




In [ ]:
print(df.loc[19, 'Website'])

In [ ]:
paragraph = ''
for x in soups[20].find_all('table'):
    for y in x.find_all(text=True):
        paragraph += y.strip().replace('\xa0', '') +' '


results= re.findall(r'- Long Beach  ([0-9]+)', paragraph)
# print(results)
cases = results[0]
deaths = results[1]
 

In [ ]:
df.loc[19, name] = cases
df.loc[19, string_] = deaths

## Los Angeles 
data sometimes includes Long Beach and Pasadena Counties

In [ ]:
county = 20


In [ ]:
print(df.loc[20, 'Website'])

In [ ]:
paragraph = ''
for x in soups[20].find_all('table'):
    # print(x.find_all('th'))
    for te, num in zip(x.find_all('th'), x.find_all('td')):
        # print(te.find(text=True), num.find(text=True))
        paragraph += num.find(text=True).strip().replace('\xa0', '') +' '
        paragraph += te.find(text=True).strip().replace('\xa0', '') + ' '


cases= re.findall(r'Total Cases ([0-9]+)*', paragraph)[0]
deaths = re.findall(r'Deaths  \- Los Angeles County \(excl.LBandPas\) ([0-9]+)', paragraph)

In [ ]:
df.loc[20, name] = cases
df.loc[20, string_] = deaths

## MADERA

In [ ]:
county = 21
df.loc[county]

In [ ]:
print(df.loc[21, 'Website'])

In [ ]:
info = []
paragraph = ''
for x in soups[21].find_all('div', id='widget_685_4225_1649'):
    text = x.find_all('td')
    for y in x.find_all('td'):
        paragraph += y.find(text=True).strip()  + " "
        try:
            if int(y.find(text=True)):
                info.append(str(y.find(text=True)))
        except:
            pass

In [ ]:
paragraph

In [ ]:
for no, x in enumerate(info):
    x = x.replace('\xa0', "")
    info[no] = int(x)

In [ ]:
cases = max(info)

In [ ]:
deaths = re.findall(r'deceased', paragraph)
recover = re.findall(r'recovered', paragraph)
len(deaths), len(recover)

In [ ]:
df.loc[21, name] = cases
df.loc[21, string_] = len(deaths)
df.loc[21, recovered] = len(recover)


## MARIN

In [ ]:
county = 22
df.loc[county]

In [ ]:
print(df.loc[county, 'Website']) 

In [ ]:
# soups[22] = run_bs4(df.loc[22, 'Website'])

In [ ]:
info=[]
for x in soups[22].find_all('table', class_="table table-striped table-hover cols-1"):
    # print(x)
    for y in x.find_all('tbody'):
        # print(y.find_all(text=True))
        for no, z in enumerate(y.find_all(text=True)):
             if len(z) >1:
                    info.append(z.strip())
   

In [ ]:
paragraph = regex_paragraph_1(info)
paragraph

In [ ]:
paragraph = regex_paragraph_1(info)
cases = re.findall(r'Total Cases ([0-9]+)', paragraph)[0]

cases

In [ ]:
df.loc[22, name] = cases
df.loc[22, string_]  = 4
df.loc[22, test_admin] = 716

## MARIPOSA

In [ ]:
df.loc[23, 'Website'] 

In [ ]:
info = []
for x in soups[23].find_all('div', class_="fr-view"):
    for y in x.find_all('strong'):
        for z in y.find_all(text=True):
            info.append(str(z).replace('\xa0', ""))

In [ ]:
paragraph = ''
for x in info:
    paragraph += x + ' '
paragraph

In [ ]:
cases = re.search(r'Total Positive:   [0-9]+', paragraph).group()
cases = re.search(r'[0-9]+', cases).group()

In [ ]:
cases = re.findall(r'Total Positive:   ([0-9]+)', paragraph)[0]
tests = re.findall(r'Total Tested:   ([0-9]+)', paragraph)[0]
deaths = re.findall(r'Total Deaths:    ([0-9]+)', paragraph)[0]


In [ ]:
df.loc[23, test_admin] = tests
df.loc[23, name] = cases
df.loc[23, string_] = deaths

## MENDOCINO

In [ ]:
print(df.loc[24, 'Website']) 

In [ ]:
paragraph =  ''
for x in soups[24].find_all('div', id="widget_313_6901_4337"):
    for y in x.find_all('li'):
      
        text = y.find_all(text=True)
       
        for z in text: 
            
            paragraph += z.strip() +' '
            
pos = re.findall(r'Positive tests: ([0-9]+)', paragraph)[0]
tests = re.findall(r'Total tests: ([0-9]+)', paragraph)[0]

df.loc[24, name] = pos
df.loc[24, test_admin] = tests

## MERCED

In [ ]:
df.loc[25]

In [765]:
info={}
for x in soups[25].find_all('table'):
    print(x)
    for title, no in zip(x.find_all('th'), x.find_all('td')):
        info[title.find(text=True)] = int(no.find(text=True))

info

<table class="covidStats">
<tr>
<th>Cases</th>
<th>Travel Related</th>
<th>Community Spread</th>
<th>Under Investigation</th>
<th>Deaths</th>
</tr>
<tr>
<td>16</td>
<td>8</td>
<td>2</td>
<td>6</td>
<td>0</td>
</tr>
</table>


{'Cases': 16,
 'Travel Related': 8,
 'Community Spread': 2,
 'Under Investigation': 6,
 'Deaths': 0}

In [766]:
df.loc[25, name]= info['Cases']
df.loc[25, string_]= info['Deaths']
# df.loc[25, recovered] = info['Recoveries']
# df.loc[25, test_admin] = info['Tests']

## MODOC

In [767]:
df.loc[26, 'Website'] = 'https://www.modocsheriff.us/modoc-covid-19-incident-updates'
print(df.loc[26, 'Website'])

https://www.modocsheriff.us/modoc-covid-19-incident-updates


In [768]:
soups[26] = run_bs4('https://www.modocsheriff.us/modoc-covid-19-incident-updates')
soups[26]

<html><body><h1>403 Forbidden</h1>
Request forbidden by administrative rules.
</body></html>

In [769]:
df.loc[26, test_admin] = 31

## MONO

In [770]:
print(df.loc[27, 'Website'])

https://coronavirus.monocounty.ca.gov/


In [771]:
with open('../webpages/Mono County Coronavirus Response.htm', 'r') as htm:
    print(htm.read())

<!DOCTYPE html><html lang="en-us"><head><title>Mono County Coronavirus Response</title><meta name="twitter:title" content="Mono County Coronavirus Response"><meta property="og:title" content="Mono County Coronavirus Response"><meta name="description" content="Discover the latest resources, maps and information about the coronavirus (COVID-19) outbreak in Mono County"><meta name="twitter:description" content="Discover the latest resources, maps and information about the coronavirus (COVID-19) outbreak in Mono County"><meta property="og:description" content="Discover the latest resources, maps and information about the coronavirus (COVID-19) outbreak in Mono County"><meta name="twitter:image" content="https://gis.mono.ca.gov/images/logos/main-logo.png"><meta property="og:image" content="https://gis.mono.ca.gov/images/logos/main-logo.png"><meta name="twitter:card" content="summary"><meta property="og:url" content="https://coronavirus.monocounty.ca.gov/"><meta property="og:type" content="w

In [772]:
df.loc[27, name]= 18
df.loc[27, string_]= 1

df.loc[27, test_admin] = 84

## MONTEREY

In [773]:
print(df.loc[28, 'Website'])

https://www.co.monterey.ca.us/government/departments-a-h/administrative-office/office-of-emergency-services/response/covid-19


In [774]:
# soups[28] = run_bs4(df.loc[28, 'Website'])

In [775]:
paragraph = ''
for no_1, x in enumerate(soups[28].find_all('div')):
    for no, y in enumerate(x.find_all('p')):
        text = y.find_all(text=True)
        for z in text:
            if len(z) > 0:
                paragraph += str(z.strip()) + ' '
try: 
    cases = re.findall(r'Total ([0-9]+) 100', paragraph)[0]
    tests = re.findall(r'Number of tests completed: ([0-9]+)', paragraph)[0]
    deaths = re.findall(r'Fatalities ([0-9]+)', paragraph)[0]

except:
    cases = re.findall(r'Total ([0-9]+) 100', paragraph)[0]
    tests = re.findall(r' completado: ([0-9]+)', paragraph)[0]
    deaths = re.findall(r' mortales ([0-9]+)', paragraph)[0]

In [776]:

    
cases, tests, deaths

('48', '561', '2')

In [777]:
df.loc[28, name]= cases
df.loc[28, string_]= deaths
df.loc[28, test_admin] = tests

## NAPA

In [778]:
df.loc[29, 'Website'] = 'https://legacy.livestories.com/s/v2/coronavirus-report-for-napa-county-ca/9065d62d-f5a6-445f-b2a9-b7cf30b846dd/'

In [779]:
print(df.loc[29, 'Website'])

https://legacy.livestories.com/s/v2/coronavirus-report-for-napa-county-ca/9065d62d-f5a6-445f-b2a9-b7cf30b846dd/


In [780]:
!ls ../data/webpages

ls: ../data/webpages: No such file or directory


In [948]:
with open('../webpages/Coronavirus Report for Napa County, CA - LiveStories.mhtml', 'r') as htm:
    text = htm.read()

In [949]:
soups[29] = bs4.BeautifulSoup(text)

In [950]:
paragraph = ''
for x in soups[29].find_all('table'):
    for y in x.find_all('tr'):
        for z in y.find_all('td'):
            for a in z.find_all(text=True):
                paragraph += str(a.strip()).replace('\n', '').replace('=20', '').replace('         ', '')
                
info_cases = re.findall(r'Napa County   ([0-9]+)   ([0-9]+)', paragraph)
cases = info_cases[0][0]
deaths = info_cases[0][1]
tests = re.findall(r'TOTAL  ([0-9]+)', paragraph)


df.loc[29, name] = cases
df.loc[29, string_] = deaths
#df.loc[29, recovered] = recover_
tests = re.findall(r'TOTAL   ([0-9]+)', paragraph)
tests = tests[0]

df.loc[29, test_admin] = tests

## NEVADA

In [784]:
df.loc[30, 'Website'] = 'https://www.mynevadacounty.com/2924/Coronavirus'

In [785]:
# soups[30] = run_bs4(df.loc[30, 'Website'])

In [786]:
paragraph += str(a.strip()).replace('\n', '').replace('=20', '').replace('         ', '')

In [787]:
paragraph = ''
for x in soups[30].find_all('table'):
    for y in x.find_all('tr'):
        for z in y.find_all('td'):
            for a in z.find_all(text=True):
                print(a)
                paragraph += str(a.strip()) + ' '

Total Nevada County Residents
Positive Tests
26
Deaths
0
Eastern Nevada County Residents
Positive Tests
18
Deaths
0
Western Nevada County Residents
Positive Tests
8
Deaths
0


In [788]:
paragraph

'Total Nevada County Residents Positive Tests 26 Deaths 0 Eastern Nevada County Residents Positive Tests 18 Deaths 0 Western Nevada County Residents Positive Tests 8 Deaths 0 '

In [789]:
results = re.findall(r'Positive Tests ([0-9]+)',paragraph) 
cases = results[0]
cases

'26'

In [790]:
deaths= re.findall(r'Deaths ([0-9]+)', paragraph)
deaths = deaths[0]

In [791]:
df.loc[30, name] = cases
df.loc[30, string_] = deaths

## ORANGE COUNTY

In [792]:
paragraph = ''
for x in soups[31].find_all('div', class_="col-md-6 col-sm-6 col-xs-12"):
    for y, z in zip(x.find_all('h2'), x.find_all('h1')):
        paragraph += y.find(text=True) +' ' + z.find(text=True) +' '
        
df.loc[31, name]= re.findall(r'Cumulative Cases to Date ([0-9]+)', paragraph)[0]
df.loc[31, string_]= re.findall(r'Cumulative Deaths to Date ([0-9]+)', paragraph)[0]
# df.loc[31, test_admin] = info['test']
        

In [793]:
print(df.loc[31, 'Website'])

https://occovid19.ochealthinfo.com/coronavirus-in-oc


## PASADENA

In [794]:
df.loc[32, 'Website'] = 'https://www.cityofpasadena.net/public-health/news-announcements/information-on-covid-19/'

In [795]:
info={}
key=[]
for x in soups[32].find_all('aside'):
    table = x.find('table')
    rows = table.find_all('tr')

    for no, y in enumerate(rows):
        count = 0
        for a in y.find_all('th'):
            key.append(a.find(text=True))
            
        for b in y.find_all('td'):
            print(key[count])
            
            info[key[count]]=b.find(text=True)
            count+=1

df.loc[32, name]= int(info['Cases'])
df.loc[32, string_]= int(info['Deaths'])

Cases
Deaths


In [796]:
info

{'Cases': '37', 'Deaths': '0'}

## PLACER

In [797]:
df.loc[33, 'Website'] = 'https://www.placer.ca.gov/6448/Cases-in-Placer'

In [798]:
paragraph = ''

for x in soups[33].find_all('table'):
    for no, y in enumerate(x.find_all('td')):
        
        paragraph += y.find(text=True) + ' '
           
df.loc[33, name]= re.findall(r'Lab Confirmed Cases \(includes those who have died\) ([0-9]+)', paragraph)[0]
df.loc[33, string_]= re.findall(r'Deaths ([0-9]+)', paragraph)[0]         

In [799]:
paragraph

'Lab Confirmed Cases (includes those who have died) 90 Deaths 2 Community-Acquired 28% Contact with Known Case 9% While Providing Health Care 10% Travel 33% Pending 20% 0-17 years 0% 18-49 years 46% 50-64 years 33% 65+ years 21% Male 54% Female 46% '

## PLUMAS

In [800]:
 df.loc[34, 'Website']

'https://www.plumascounty.us/2669/Novel-Coronavirus-2019-COVID-19'

In [801]:
paragraph =''
key=[]
for x in soups[34].find_all('table'):
    y = x.find('tbody')
    count = 0
    for no, a in enumerate(y.find_all('td')):
        print(no, a.find(text=True))
        text = a.find(text=True)
        if text: 
            paragraph += y.find_all('td')[no].find(text=True) + ' '
            


0 COVID-19 DASHBOARD
1 DATA LAST UPDATED: Wednesday, April 1, 2020, at 5:00 PM
2 Disclaimer: Data may not be updated on Saturdays and Sundays.
0 Positive Test Results
1 1
2 People Tested
3 64
4 Pending Test Results
5 16
6 Negative Test Results
7 47
0 State of CA
1 6,932
2 150
3 U.S.
4 186,101 
5 3,603  
0 SITUATION UPDATE
0 Information Line: (530) 283-6400


In [802]:
paragraph

'COVID-19 DASHBOARD DATA LAST UPDATED: Wednesday, April 1, 2020, at 5:00 PM Disclaimer: Data may not be updated on Saturdays and Sundays. Positive Test Results 1 People Tested 64 Pending Test Results 16 Negative Test Results 47 State of CA 6,932 150 U.S. 186,101\xa0 3,603 \xa0 SITUATION UPDATE Information Line: (530) 283-6400 '

In [803]:
results = re.findall(r'Positive Test Results ([0-9]+) People Tested ([0-9]+)', paragraph)
results

[('1', '64')]

In [804]:
cases = results[0][0]
tests = results[0][1]

In [805]:
df.loc[34, name]= cases

df.loc[34, test_admin] = tests

## Riverside

In [806]:
print(df.loc[35, 'Website'])

https://www.rivcoph.org/coronavirus


In [807]:
paragraph = ''
for x in soups[35].find_all('div'):
    # print(x)
    for y in x.find_all('p'):
        #print(y.find_all(text=True))
        
        for no, z in enumerate(y.find_all(text=True)):
                paragraph += z.strip().replace('\xa0', '') + ' '
       

In [808]:
'''
cases = re.findall(r'Confirmed cases : ([0-9]+)', paragraph)
cases = cases[0]
deaths = re.findall(r'Deaths: ([0-9]+)', paragraph)
deaths = deaths[0]
deaths, cases
'''

"\ncases = re.findall(r'Confirmed cases : ([0-9]+)', paragraph)\ncases = cases[0]\ndeaths = re.findall(r'Deaths: ([0-9]+)', paragraph)\ndeaths = deaths[0]\ndeaths, cases\n"

In [809]:
df.loc[35, name]= 493
df.loc[35, string_]= 14


## San Benito

In [810]:
# manually check dashboard
print(df.loc[36, 'Website']) 

https://hhsa.cosb.us/publichealth/communicable-disease/coronavirus/


In [811]:
df.loc[36, name]= 23
df.loc[36, string_]= 1
df.loc[36, test_admin] = 289
df.loc[36, recovered] = 11

## San Bernardino

In [812]:
print(df.loc[37, 'Website']) 

http://wp.sbcounty.gov/dph/coronavirus/


In [813]:
paragraph = ''
for x in soups[37].find_all('div', class_= "et_pb_text_inner"):
   
    for z, y in zip(x.find_all('h2'), x.find_all('span')):
       
           paragraph += z.find(text=True) + ' '+ y.find(text=True) + ' '
        
cases = re.findall(r'COVID-19 CASES IN ([0-9]+)', paragraph)
deaths = re.findall(r'COVID-19 ASSOCIATED DEATHS IN ([0-9]+)', paragraph)
cases, deaths

(['254'], ['6'])

In [814]:
df.loc[37, name]= re.findall(r'COVID-19 CASES IN ([0-9]+)', paragraph)[0]
df.loc[37, string_]= re.findall(r'COVID-19 ASSOCIATED DEATHS IN ([0-9]+)', paragraph)[0]

## SAN DIEGO



In [815]:
df.loc[38, 'Website'] 

'https://www.sandiegocounty.gov/content/sdc/hhsa/programs/phs/community_epidemiology/dc/2019-nCoV/status.html'

In [816]:
    # dictionary_outer keys : ['COVID-19 Case Summary', 'San Diego County Residents',
    #                           'Non-SanDiego County Residents', 'Total']
    # dictionary_inner sample : info ['Total'] =
    # {'Total Positives': '297','0-9 years': '2','10-19 years': '3',
    #                          '20-29 years': '65','40-49 years': '54',
    #                          '50-59 years': '40','60-69 years': '24',
    #                          '70-79 years': '20','80+ years': '12',
    #                          'Age Unknown': '0','Gender': '\xa0',
    #                          'Female': '112','Male': '185','Unknown': '0',
    #                          'Hospitalizations': '59', 'Intensive Care': '29',
    #                          'Deaths': '2'}
'''
key = []
    info = {}
    for x in soup.find_all('table'):
        for no_1, y in enumerate(x.find_all('tr')):
            text = y.find_all(text=True)

            if '\n' in text:
                key.append(text)

    # trying to undo some wonky formatting
    for no, item in enumerate(key[1]):
        key[1][no] = str(item).replace('\n', '').replace("     ", "")

    # trying to remove some unicode characters from strings
    for no, x in enumerate(key[3]):
        key[3][no]=key[3][no].replace('\xa0', '0')

    # setting up some initial key values for dictionary
    list_=[]
    error_ = []
    info[key[1][1]] = {key[1][1] : list_}
    info[key[1][3]] = {key[2][1]:key[2][3]}
    info[key[1][5]] = {key[2][1]:key[2][5]}
    info[key[1][7]] = {key[2][1]:key[2][7]}

    for x in range(2,21):
        try:
            list_.append(key[x][1])
            info[key[1][3]].update({key[x][1]: key[x][3]})
            info[key[1][5]].update({key[x][1]: key[x][5]})
            info[key[1][7]].update({key[x][1]: key[x][7]})
        except:
            error_.append(x)



    cases = info['Total']['Total Positives']
    deaths = info['Total']['Deaths']
    return((cases, deaths, 0,0))'''

'\nkey = []\ninfo = {}\nfor x in soup.find_all(\'table\'):\n    for no_1, y in enumerate(x.find_all(\'tr\')):\n        text = y.find_all(text=True)\n\n        if \'\n\' in text:\n            key.append(text)\n\n# trying to undo some wonky formatting\nfor no, item in enumerate(key[1]):\n    key[1][no] = str(item).replace(\'\n\', \'\').replace("     ", "")\n\n# trying to remove some unicode characters from strings\nfor no, x in enumerate(key[3]):\n    key[3][no]=key[3][no].replace(\'\xa0\', \'0\')\n\n# setting up some initial key values for dictionary\nlist_=[]\nerror_ = []\ninfo[key[1][1]] = {key[1][1] : list_}\ninfo[key[1][3]] = {key[2][1]:key[2][3]}\ninfo[key[1][5]] = {key[2][1]:key[2][5]}\ninfo[key[1][7]] = {key[2][1]:key[2][7]}\n\nfor x in range(2,21):\n    try:\n        list_.append(key[x][1])\n        info[key[1][3]].update({key[x][1]: key[x][3]})\n        info[key[1][5]].update({key[x][1]: key[x][5]})\n        info[key[1][7]].update({key[x][1]: key[x][7]})\n    except:\n        e

In [817]:
key = []
key_2 = []
info = {}
paragraph = ''
for x in soups[38].find_all('table'):
    for no_1, y in enumerate(x.find_all('tr')):
        for z in  y.find_all('td'):
            paragraph += z.find(text=True).strip().replace('\xa0', '').replace('\n', '') + ' '
        
        



In [818]:
paragraph

' COVID-19 Case Summary San Diego County      Residents Total Positives 849 Age Groups  0-9 years 6 10-19 years 8 20-29 years 163 30-39 years 194 40-49 years 154 50-59 years 125 60-69 years 93 70-79 years 62 80+ years 41 Age Unknown 3 Gender  Female 380 Male 461 Unknown 8 Hospitalizations 158 Intensive Care 65 Deaths 15 '

In [819]:
cases= re.findall(r'Residents Total Positives ([0-9]+)', paragraph)[0]
deaths= re.findall(r'Deaths ([0-9]+)', paragraph)[0]

In [820]:
df.loc[38, name]= cases
df.loc[38, string_]= deaths

## SAN FRANCISCO







In [821]:
df.loc[39, 'Website'] 

'https://www.sfdph.org/dph/alerts/coronavirus.asp'

In [822]:
paragraph = ''
for x in soups[39].find_all('div', class_='box2'):
    for y in x.find_all('p'):
        paragraph += y.find(text=True) +' '

In [823]:
paragraph




'Total Positive Cases: 450 Deaths: 7 updated daily at 9:00 AM '

In [824]:
cases = re.findall(r'Total Positive Cases: ([0-9]+)', paragraph)[0]
deaths = re.findall(r'Deaths: ([0-9]+)', paragraph)[0]
deaths

'7'

In [825]:
df.loc[39, name]= cases
df.loc[39, string_]= deaths

## San Joaquin





In [826]:
df.loc[40, 'Website']

'http://www.sjcphs.org/coronavirus.aspx'

In [827]:
paragraph = ''
for x in soups[40].find_all('tr'):
    for y in x.find_all('strong'):
        paragraph += y.find(text=True) + ' '
       

In [828]:
paragraph

'Confirmed COVID-19 Cases 173 9 '

In [829]:
cases = re.findall(r'Confirmed COVID-19 Cases ([0-9]+) ([0-9]+)', paragraph)
cases

[('173', '9')]

In [830]:
df.loc[40, name]= cases[0][0]
df.loc[40, string_]= cases[0][1]

## San Luis Opisbo


In [831]:
print(df.loc[41, 'Website']) 

https://www.emergencyslo.org/en/positive-case-details.aspx


In [832]:
page = requests.get('https://e.infogram.com/f6d9f731-5772-4da5-b149-5e42cc1c3b89?parent_url=https%3A%2F%2Fwww.emergencyslo.org%2Fen%2Fpositive-case-details.aspx&src=embed#')

In [833]:
slo = pd.read_csv('../data/data_slo.csv', header=None)


In [834]:
slo

,0,1,2
0,"<p><span style=""font-size: 40px"">80</span></p>","<font style=""font-size: 25px"">Confirmed cases<...",positivePath
1,"<span style=""font-size: 40px"">40</span>","<font style=""font-size: 25px"">Home</font>",1144Path
2,"<span style=""font-size: 40px"">30</span>","<font style=""font-size: 25px"">Recovered</font>",1250234Path
3,"<span style=""font-size: 40px"">10</span>","<font style=""font-size: 25px"">Hospitalized (2 ...",147030Path
4,"<font style=""font-size: 40px"">0</font>","<font style=""font-size: 25px"">Deaths</font>",rectPath


In [835]:


for no, y in enumerate(slo.loc[:, 0]):
    if no == 0:
        cases = re.findall(r'>([0-9]+)<', y)[0]
    
    if no == 2:
        recovered_ = re.findall(r'40px">([0-9]+)<', y)[0]
        
    if no == 3:
        deaths = re.findall(r'40px">([0-9]+)<', y)[0]

In [836]:
df.loc[41, name]= cases
df.loc[41, string_]= deaths
df.loc[41, recovered]= recovered_

In [837]:
soups[41]=run_bs4('https://e.infogram.com/f6d9f731-5772-4da5-b149-5e42cc1c3b89?parent_url=https%3A%2F%2Fwww.emergencyslo.org%2Fen%2Fpositive-case-details.aspx&src=embed#async_embed')

## San Mateo

In [838]:
print(df.loc[42, 'Website']) 

https://www.smchealth.org/coronavirus


In [839]:
paragraph = ''
for x in soups[42].find_all('table'):
    print(x)
    for y in x.find_all('td'):
        paragraph += y.find(text=True).strip() + ' '

In [840]:
paragraph

''

In [841]:
# results = re.findall(r' ([0-9]+) Deaths ([0-9]+)', paragraph)

cases = 309
deaths = 10

In [842]:
df.loc[42, name]= cases
df.loc[42, string_]= deaths


## Santa Barbara

In [951]:
with open('../webpages/Status Reports – County of Santa Barbara.mhtml', 'r') as htm:
    soups[43] = htm.read()

In [952]:
print(df.loc[43, 'Website'])
    

https://publichealthsbc.org/status-reports/


In [953]:
work_around = bs4.BeautifulSoup(soups[43])

In [954]:
info =[]
dates = []
for x in work_around.find_all('div'):
    for y,z in zip(x.find_all('td'), x.find_all('li')):
        nums = y.find_all(text=True)
        text = z.find_all(text=True)
        info.append(nums)
        if 'As of' in str(text):
            info.append(text)
        
paragraph = ''
for x in info:
    for y in x: 
        paragraph += y + ' '   
pos = re.findall(r'# Positive Results ([0-9]+)', paragraph)[0]
rec = re.findall(r'Recovered ([0-9]+)', paragraph)[0]
tests = re.findall(r'Total Tested ([0-9]+)', paragraph)[0]
df.loc[43, name]= pos
df.loc[43, test_admin]= tests
df.loc[43, recovered] = rec

In [956]:
pos

'111'

In [957]:

rec

'2'

In [958]:

tests

'493'

## Santa Clara

In [852]:
df.loc[44, 'Website'] = 'https://www.sccgov.org/sites/phd/DiseaseInformation/novel-coronavirus/Pages/home.aspx'

In [853]:
print(df.loc[44, 'Website'])

https://www.sccgov.org/sites/phd/DiseaseInformation/novel-coronavirus/Pages/home.aspx


In [854]:
paragraph = ''
for x in soups[44].find_all('table'):
    # print(x)
    for no, y in enumerate(x.find_all('td')):
        for z in y.find_all(text=True):
            paragraph += z.replace('\n', '') + ' '
        
        
#paragraph += 

In [855]:
info = re.findall(r"Total Confirmed Cases Hospitalized Deaths ([0-9]+) ([0-9]+) ([0-9]+)", paragraph)

In [856]:
info

[]

In [857]:
df.loc[44, name] = 1019
df.loc[44, string_] = 36
df.loc[44, test_admin] = 9218


## Santa Cruz


In [858]:
df.loc[45, 'Website'] 

'http://www.santacruzhealth.com/HSAHome/HSADivisions/PublicHealth/CommunicableDiseaseControl/Coronavirus.aspx'

In [859]:
paragraph = ''
for x in soups[45].find_all('div', class_='Normal'):
    for y in x.find_all('td'):
        text = y.find_all('p')
        for z in y.find_all('p'):
            paragraph += z.find(text=True).replace('\xa0', "") + ' '
            
results = re.findall(r'Cases/Deathsas of 0[0-9]/[0-9]+/20 [0-9]+:[0-9]+[ap]m ([0-9]+)/ ([0-9]+)', paragraph)
results
cases = results[0][0]
deaths=results[0][1]

In [860]:
paragraph

'Verified Cases/Deathsas of 04/02/20 10:53am 57/ 1 3/31/20 UPDATED 3/29/20  3/25/20  3/26/20  3/24/20  3/23/20  3/23/20  3/23/20  3/23/20  3/22/20  3/20/20  '

In [861]:
df.loc[45, name]= cases
df.loc[45, string_] = deaths

## Shasta

In [862]:
df.loc[46, 'Website'] = 'https://www.co.shasta.ca.us/covid-19/overview'

In [863]:
info = []
for x in soups[46].find_all('table'):
    for y in x.find_all('td'):
        info.append(y.find(text=True))

In [864]:
paragraph = regex_paragraph_1(info)

In [865]:
cases = re.search(r'Total Confirmed Cases [0-9]+', paragraph).group()
cases = re.search(r'[0-9]+', cases).group()

deaths =  re.search(r'Deaths [0-9]+', paragraph).group()
deaths =  re.search(r'[0-9]+', deaths).group()

In [866]:
info

['Total Confirmed Cases',
 '7',
 'People who tested negative',
 '370',
 'Travel Related',
 '4',
 'Person-to-Person Spread ',
 '2',
 'Community Acquired ',
 '1',
 'Deaths',
 '1']

In [867]:
df.loc[46, name]= cases
df.loc[46, string_]= deaths

## SIERRA

In [868]:
df.loc[47, 'Website'] = 'http://sierracounty.ca.gov/582/Coronavirus-COVID-19'

In [869]:
soups[47] = run_bs4('http://sierracounty.ca.gov/582/Coronavirus-COVID-19')

In [870]:
info = []
for x in soups[47].find_all('table'):
    for y in x.find_all('tr'):
        for z in y.find_all(text=True):
            if '\n' not in z:
                info.append(z)
        
        

In [871]:
info

['Approximate # of Tests in Sierra County',
 'Thursday, April 2, 2020 11:00 am',
 '# of Test Administered',
 '7',
 '# of Pending Test Results',
 '0',
 '# of Negative Test Results',
 '7',
 '# of Positive COVID-19 Cases',
 '0']

In [872]:
paragraph = regex_paragraph_1(info)

In [873]:
cases = re.search(r'# of Positive COVID-19 Cases [0-9]+', paragraph).group()
cases = re.search(r' [0-9]+', cases).group()

tests =  re.search(r'# of Test Administered [0-9]+', paragraph).group()
tests =  re.search(r'[0-9]+', tests).group()
tests, cases

('7', ' 0')

In [874]:
df.loc[47, name]= cases
df.loc[47, string_]= cases
df.loc[47, test_admin]= tests

## Siskiyou

In [875]:
print(df.loc[48, 'Website'] )

http://www.co.siskiyou.ca.us/content/public-health-division


In [960]:
with open('../webpages/Public Health _ Siskiyou County California.mhtml', 'r') as htm:
    soups[48] = htm.read()

In [961]:
soups[48] = bs4.BeautifulSoup(soups[48])

In [965]:
info = []
for x in soups[48].find_all('table'):
    for y in x.find_all('td'):
        
        for z in y.find_all(text=True):
            if z != '\n':
                info.append(z)
paragraph = regex_paragraph_1(info[:12])
paragraph

'3 TOTAL CONFIRMED POSITIVE CASES 0 TOTAL ACTIVE CONFIRMED POSITIVE CASES 80 TOTAL NEGATIVE TESTS 10 TOTAL PENDING TESTS 0 TOTAL DEATHS IN SISKIYOU COUNTY 93 TOTAL TESTED IN SISKIYOU COUNTY '

In [969]:

cases = re.findall(r'([0-9]+) TOTAL CONFIRMED POSITIVE', paragraph)[0]

tests =  re.findall(r'([0-9]+) TOTAL TEST', paragraph)[0]

deaths =  re.findall(r'([0-9]+) TOTAL DEATHS', paragraph)[0]
df.loc[48, name]= cases
df.loc[48, string_]= deaths
df.loc[48, test_admin]= tests

## Solano

In [880]:
df.loc[49, 'Website'] = 'http://www.solanocounty.com/depts/ph/ncov.asp'

In [970]:
print(df.loc[49, 'Website'])

http://www.solanocounty.com/depts/ph/ncov.asp


In [881]:
run_bs4('https://admin.solanocounty.com:4433/civicax/filebank/blobdload.aspx?BlobID=31871');

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [974]:
df.loc[49, name] = 61
df.loc[49, string_] = 1

## Sonoma

In [884]:
print(df.loc[50, 'Website'])

https://socoemergency.org/


In [975]:
with open('../webpages/Novel Coronavirus - Sonoma County Emergency and Preparedness Information.mhtml', 'r') as htm:
    soups[50] = htm.read()

In [976]:
soups[50] = bs4.BeautifulSoup(soups[50])

In [977]:
info=[]
for x in soups[50].find_all('div'):
    for y in x.find_all('p'):
        for z in y.find_all(text=True):
            if len(z) >0:
                info.append(z)

In [978]:
paragraph = regex_paragraph_1(info[1:11])
cases = re.findall(r'Total Cases ([0-9]+)', paragraph)[0]
tests =  re.findall(r'Tests ([0-9]+)', paragraph)[0]

deaths =  re.findall(r'Deaths ([0-9]+)', paragraph)[0]

recovered_ = re.findall(r'Recovered ([0-9]+)', paragraph)[0]


cases, deaths, tests, recovered_

('95', '1', '1915', '15')

In [979]:
df.loc[50, name]= cases
df.loc[50, string_]= deaths
df.loc[50, test_admin]= tests
df.loc[50, recovered]= recovered_


## Stanislaus

In [890]:
df.loc[51, 'Website'] = 'http://www.schsa.org/PublicHealth/pages/corona-virus/'


In [891]:
# soups[51]= run_bs4('http://www.schsa.org/PublicHealth/pages/corona-virus/')

In [892]:
paragraph = ''
for z in soups[51].find_all('p'):
    for b in z.find_all('strong'):
        if 'Cases' in str(b):
            paragraph += (b.find(text=True))+ " "
        if 'Tests' in str(b):
            paragraph += (b.find(text=True))+ " "
        if 'Deaths' in str(b):
            paragraph += (b.find(text=True))+ " "

for x in soups[51].find_all('div', class_='counter'):
    for y in x.find_all(text=True): 
        print(y.strip())
        if len(y) >1:
            paragraph += y.strip()+ " "
            no +=1
            
results = re.findall(r'Positive Cases Negative Tests Related Deaths ([0-9]+) ([0-9]+) ([0-9])+', paragraph)

cases = results[0][0]
tests = int(results[0][1]) + int(results[0][0])
deaths = results[0][2]


50


1337


0



In [893]:
df.loc[51, name]= cases
df.loc[51, string_]= deaths
df.loc[51, test_admin]= tests


## Sutter

In [894]:
print(df.loc[52, 'Website']) 

https://www.suttercounty.org/doc/government/depts/cao/em/coronavirus


In [895]:
# soups[52] = run_bs4(df.loc[52, 'Website'])

In [896]:
paragraph = ' '
for x in soups[52].find_all('table'):
    for y in x.find_all(text=True):
        y = y.replace('\n', "")
        if len(y) > 0:
            paragraph += y.replace('\n', "") + ' '
            
for x in soups[52].find_all('p'):
    if 'tests' in str(x):
        paragraph += x.find(text=True) + ' ' 


tests = re.findall(r'reported ([0-9]+) COVID-19 tests', paragraph)

tests = tests[0]    
results = re.findall(r'Confirmed ([0-9]+) Deceased ([0-9]+)', paragraph)
tests, results

('348', [('14', '1'), ('8', '0')])

In [897]:
paragraph

' Confirmed 22 Hospitalized 3 Confirmed 14 Deceased 1 Confirmed 8 Deceased 0 \nThe Yuba-Sutter Public Health Department released the following information about testing rates in Yuba and Sutter counties. As of today, Sunday, March 29, 2020, a reported 348 COVID-19 tests have been administered between those authorized by Public Health and those performed at private labs, with 215 results received and 133 tests pending. Out of the 215 results received, there have been 10 positives and 205 negatives. Because there is only a requirement to report confirmed cases to Public Health, there is long lag time in getting information about all completed tests. The reported numbers will always lag behind the actual number of tests.\n '

In [898]:
cases = results[0][0]
deaths = results[0][1]

In [899]:
df.loc[52, name] = cases
df.loc[52, test_admin] = tests
df.loc[59, name] = results[0][1]

## Tehama

In [900]:
print(df.loc[53, 'Website']) 

https://www.tehamacohealthservices.net/news-events/news/


##  Trinty

In [901]:
print(df.loc[54, 'Website'])

https://www.trinitycounty.org/Public-Health


## Tulare

In [902]:
print(df.loc[55, 'Website'])

https://tchhsa.org/eng/index.cfm/public-health/covid-19-updates-novel-coronavirus/


In [903]:
soups[55] = run_bs4('https://tchhsa.org/eng/index.cfm/public-health/covid-19-updates-novel-coronavirus/')

In [980]:
with open('../webpages/English - COVID-19 (Novel Coronavirus).mhtml', 'r') as htm:
    soups[55] = htm.read()
soups[55] = bs4.BeautifulSoup(soups[55])

In [981]:
paragraph = ""
for x in soups[55].find_all('div', class_= '3D"col-6'):
    for y in  x.find_all(text=True):
        if len(y) > 0:
            paragraph += y.strip().replace('=\n', '') + " "

In [982]:
paragraph

' 74 - Total positive cases  17= - Travelers  39 - Person to Person/Contact &nbsp; 18 - Unknown/Under Investigation &nbsp; 4 - Recovered Cases  2 - Deaths  178 - Under self-quarantine and being monitored by Tulare County Public Health officials. '

In [983]:
cases = re.findall(r'([0-9]+) - Total positive cases', paragraph)[0]

In [984]:
deaths = re.findall(r'([0-9]+) - Deaths', paragraph)[0]
recovered_ = re.findall(r'([0-9]+) - Recovered Cases', paragraph)[0]

In [985]:
df.loc[55, name]= cases
df.loc[55, string_] = deaths
df.loc[55, recovered] = recovered_



## Tuolumne

In [910]:
print(df.loc[56, 'Website'])

https://www.tuolumnecounty.ca.gov/250/Public-Health


In [911]:
paragraph = ''
for x in soups[56].find_all('table'):
    # print(x)
    for y, z in zip(x.find_all('th'),  x.find_all('td')):
        for a in z.find_all(text=True):
            paragraph += a.strip() +' '
            
        for b in y.find_all(text=True):
            paragraph += b.strip() +' '
        

In [912]:
paragraph

'216  TOTAL TESTED 10  TOTAL PENDING 205  TOTAL NEGATIVE 1 TOTAL POSITIVE 0 TOTAL DEATHS 1 TOTAL KNOWN NON-RESIDENT CASES '

In [913]:
cases = re.findall(r'([0-9]+) TOTAL POSITIVE', paragraph)[0]
tests =  re.findall(r'([0-9]+)  TOTAL TESTED', paragraph)[0]

deaths =  re.findall(r'([0-9]+) TOTAL DEATHS', paragraph)[0]




cases, deaths, tests

('1', '0', '216')

In [914]:
df.loc[56, name]= cases
df.loc[56, string_]= deaths
df.loc[56, test_admin]= tests

## VENTURA

In [915]:
df.loc[57, 'Website'] = 'https://www.vcemergency.com/'

In [916]:
# soups[57] = run_bs4('https://www.vcemergency.com/')

In [917]:
info = []
for x in soups[57].find_all('table'):
    for y in x.find_all('td'):
        info.append(y.find(text=True).replace(',', ''))
                        

In [918]:
paragraph = regex_paragraph_1(info)

In [919]:
paragraph

'Summary Data # New Cases 11 TOTAL CASES 160 Recovered Cases 31 Ever 34 Active Cases Under Quarantine 124 DEATHS 5 People Tested as of March 31st 2987 Confirmed Cases # Travel Related 9 Person-to-person acquired 34 Community acquired 32 Under investigation 85 Total 160 Cases by Age # Age 0-17 3 Age 18-24 14 Age 25-44 36 Age 45-64 65 Age 65+ 42 Unknown 0 Total 160 City # Camarillo 24 Moorpark 12 Ojai 2 Oxnard 24 Port Hueneme 2 Santa Paula 2 Simi Valley 27 Thousand Oaks 33 Ventura 18 Unincorporated Area – West 2 Unincorporated Area – East 14 Total 160 City/Unincorporated Area # Camarillo 24 Lake Sherwood 2 Moorpark 12 Newbury Park 3 Oak Park 7 Oak View 1 Ojai 2 Oxnard 24 Piru 1 Port Hueneme 2 Santa Paula 2 Santa Rosa Valley 1 Simi Valley 27 Somis 1 Thousand Oaks 33 Ventura 18 Total 160 '

In [920]:
df.loc[57, name]= re.findall(r'TOTAL CASES ([0-9]+)', paragraph)[0]
df.loc[57, string_]= re.findall(r'DEATHS ([0-9]+)', paragraph)[0]
df.loc[57, recovered]= re.findall(r'Recovered Cases ([0-9]+)', paragraph)[0]
df.loc[57, test_admin]= re.findall(r'Tested as of [A-Z][a-z]+ [0-9]+[a-z]+ ([0-9]+)', paragraph)[0]


## YOLO

In [921]:
print(df.loc[58, 'Website'] )

https://www.yolocounty.org/health-human-services/adults/communicable-disease-investigation-and-control/novel-coronavirus-2019


In [922]:
df.loc[58, name] = 24
df.loc[58, string_] = 1

## Yuba

In [923]:
df.loc[59, 'Website'] = 'https://www.yuba.org/coronavirus/'

In [924]:
soups[59] = run_bs4('https://www.yuba.org/coronavirus/')

In [925]:
paragraph = ' '
for x in soups[52].find_all('table'):
    for y in x.find_all(text=True):
        y = y.replace('\n', "")
        if len(y) > 0:
            paragraph += y.replace('\n', "") + ' '
            
for x in soups[52].find_all('p'):
    if 'tests' in str(x):
        paragraph += x.find(text=True) + ' ' 


tests = re.findall(r'reported ([0-9]+) COVID-19 tests', paragraph)

tests = tests[0]    
results = re.findall(r'Confirmed ([0-9]+) Deceased ([0-9]+)', paragraph)
cases = results[1][0]
deaths = results[1][1]


In [926]:
df.loc[59, name] = cases
df.loc[59, string_] = deaths
df.loc[59, test_admin] = tests

In [930]:
# GOOD LINK TO DOUBLE CHECK DATA COLLECTED
'https://www.latimes.com/projects/california-coronavirus-cases-tracking-outbreak/'

In [ ]:
df.to_csv("../output_files/CA_COUNTY_COVID"+today+'.csv', index=False)

In [935]:
! ls ../data

CA_Coronovirus_Websites_by_county.csv data_slo.csv
california_counties_FIP.csv           la_times_Ap_2.csv


In [938]:
df_la=pd.read_csv('../data/la_times_Ap_2.csv', delimiter='\t')

In [986]:
la_count = df_la.merge(df[['County',name, string_]],how='right', right_on = 'County', left_on='County')


In [987]:
la_count

,County,Deaths,Cases,2020-04-02 Confirmed Cases,2020-04-02 Deaths
0,Los Angeles,80.0,"4,071",3869,78
1,Santa Clara,36.0,"1,019",1019,36
2,San Diego,16.0,966,849,15
3,Orange,13.0,656,656,13
4,Riverside,14.0,493,493,14
5,San Mateo,10.0,453,309,10
6,San Francisco,7.0,450,450,7
7,Alameda,9.0,396,373,9
8,Contra Costa,3.0,276,276,3
9,San Bernardino,6.0,254,254,6


In [928]:
df[name]

0      264
1      NaN
2        3
3       23
4       10
5        3
6        1
7      276
8        3
9       18
10      45
11       2
12      28
13      43
14       9
15     155
16       5
17     NaN
18     NaN
19     139
20    3869
21      25
22     108
23       0
24       4
25      16
26     NaN
27      18
28      48
29      15
30      26
31     656
32      37
33      90
34       1
35     493
36      23
37     254
38     849
39     450
40     173
41      80
42     309
43      88
44    1019
45      57
46       7
47       0
48       3
49      31
50      66
51      50
52      14
53     NaN
54     NaN
55      45
56       1
57     160
58      24
59       8
Name: 2020-04-02 Confirmed Cases, dtype: object

In [988]:
df[name].fillna(0).astype(int).sum()

10344